# Image classification model

We will start with creating an image classification model which we will later use to build an object detection model. At this stage we will use tensorflow. 

In [ ]:
# Imports
import tensorflow as tf
import numpy as np
import pandas as pd
from tf import keras
from tf.keras import layers
from tf.keras.models import Sequential
import os